# Loading COVID-19 data into Beneath

To use this notebook:

1. Install the beneath client with `pip install beneath`
2. Issue a command-line secret from your settings page in the Beneath Terminal
3. Authenticate your command-line by running `beneath auth SECRET`
4. Create a project for your data using `beneath project create USERNAME/PROJECT`
5. Enter your username and project name in the `username` and `project_name` variables below

### Config

In [ ]:
username = "bem" # change to your username
project_name = "covid19" # change to your project name
stream_name = "cases"

### Create stream on Beneath

In [ ]:
import beneath
client = beneath.Client()

In [ ]:
stream_path = f"{username}/{project_name}/{stream_name}"
stream = await client.stage_stream(stream_path, """
    type Case @stream() @key(fields: ["country", "date"]) {
        country: String!
        date: Timestamp!
        cases: Int!
        deaths: Int!
    }
""")

### Scrape COVID-19 data

In [ ]:
from datetime import datetime, timezone
import requests

In [ ]:
req = requests.get("https://opendata.ecdc.europa.eu/covid19/casedistribution/json/")
json = req.json()
records = [{
        "country": record["countriesAndTerritories"],
        "date": datetime(
            year=int(record["year"]),
            month=int(record["month"]),
            day=int(record["day"]),
            tzinfo=timezone.utc
        ),
        "cases": int(record["cases"]),
        "deaths": int(record["deaths"]),
    }
    for record in json["records"]
]

### Write to Beneath

In [ ]:
await stream.write(records)

### Read from Beneath

In [ ]:
import beneath
client = beneath.Client()
df = await client.easy_read(f"{username}/{project_name}/{stream_name}")
df